In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from scipy.optimize import curve_fit
from scipy.special import voigt_profile
import importlib
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset


plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({
    "font.size": 14,          # Base font size
    "axes.titlesize": 14,     # Title
    "axes.labelsize": 14,     # Axis labels
    "xtick.labelsize": 12,    # X ticks
    "ytick.labelsize": 12,    # Y ticks
    "legend.fontsize": 12,    # Legend
    "figure.titlesize": 14
})

In [4]:
file = "Temperature_Runs.xlsx"
sheets = ["77.4","112","147","182","217"]
cols = [
    ["77_0", "77_1"],
    ["112_0", "112_1"],
    ["147_0", "147_1"],
    ["182_0", "182_1"],
    ["217_0", "217_1"]
]

start_index = 9830

In [5]:
#helper functions

def reduced_chi_squared(xdata, ydata, yerror, model, model_fit_parameters):
    dof = len(ydata)-len(model_fit_parameters)
    theoretical = model(xdata,*model_fit_parameters)
    chi2 = np.sum(((ydata-theoretical)/yerror)**2)
    return chi2/dof

def wavelength_extraction(x, start_index):
    A = 0.9958
    alpha_A = 0.0001

    B = 9096.6280
    alpha_B = 0.0289

    wavelengths = []
    uncertainties = []

    for i in range(len(x)):
        wavelength = (start_index - 0.242*x[i] - B) / A
        uncertainty = np.abs((x[i] - (B + alpha_B)) / (A + alpha_A) - wavelength)

        wavelengths.append(wavelength)
        uncertainties.append(uncertainty)

    return wavelengths

def voigt_model_double(x, A1, center1, gamma1, sigma,
                                 A2, center2, gamma2, offset):
    return (
            A1 * voigt_profile(x - center1, sigma, gamma1) +
            A2 * voigt_profile(x - center2, sigma, gamma2) +
            offset
        )